In [1]:
import os

import pandas as pd

from os.path import commonprefix
from pathlib import Path
from IPython.display import HTML

from scripts.data_processing import obtain_data
from scripts.fasta_processing import plain_to_fasta
from highlight import highlight_intron_in_seq
from fasta_processing import read_single_fasta, dict_align_to_fasta, read_fasta, dict_align_to_fasta_upd
from datasets import select_all_phylas, download_all_files_ncbi, check_transcript_count, update_data_for_species
from taxonomy_processing import create_taxonomy
from data_processing import analyze_exons, create_cassette, concat_cassette, dict_align_create, find_codon, \
    dict_align_info_analyze, dict_align_update_keys
from build_rna_structures import run_rnafold_with_highlight

## Введение

In [2]:
# читаем таблицы: с gene_id и taxonomy

column_names = ["tax_id", "org_name", "gene_id", "current_id", "status", "symbol", "aliases", "description",
                "other_designations", "map_location", "chromosome", "genomic_nucleotide_accession.version",
                "start_position_on_the_genomic_accession", "end_position_on_the_genomic_accession", "orientation",
                "exon_count", "to_delete_1", "to_delete_2"]

df = pd.read_csv("../all_nxf1_2.txt", sep="\t", skiprows=1, names=column_names, index_col=0)
df.drop(["to_delete_1", "to_delete_2"], axis=1, inplace=True)

df_taxonomy = pd.read_csv("../taxonomy/all_phylas_taxonomy.tsv", sep="\t", names=["taxid", "taxonomy"], index_col=0,
                          dtype={"taxid": int, "taxonomy": str})

# df_Deuterostomia = df_taxonomy[df_taxonomy.taxonomy.str.contains("Deuterostomia")]  # 436 видов, из которых 238 млекопитающие
# df_Deuterostomia.to_csv(f"../Deuterostomia_taxonomy.tsv", sep="\t", header=False)
# create_taxonomy(f"../Deuterostomia_taxonomy.tsv")

## Переопределение функций

In [3]:
def create_taxids(df, sub_phylum_list):
    taxids = {}
    for sub_phylum in sub_phylum_list:
        df_subset = df[df.taxonomy.str.contains(sub_phylum)]
        taxids[sub_phylum] = df_subset.index.tolist()
    return taxids

def create_many_cassettes(phylum: str, data: dict) -> dict:
    introns = {}
    for org_name, (df, exons_i) in data.items():
        cassette = create_cassette(phylum, org_name, df, exons_i=exons_i)
        introns[org_name] = concat_cassette(cassette, "i")
    return introns

def find_cassettes(phylum):
    prefix = "../datasets"
    postfix = "ncbi_dataset/data"
    nof = "exons.fa"
    ref_exon_len = [37]
    found_org_name_i = {}
    data = {}

    for org_name_i in os.listdir(f"{prefix}/{phylum}"):
        df_exons = analyze_exons(f"{prefix}/{phylum}/{org_name_i}/{postfix}/{nof}")
        condition = set(ref_exon_len) & set(df_exons.length.tolist())
        if condition:
            found_org_name_i[org_name_i] = condition

    for org_name_i in found_org_name_i:  # CHANGE
        df_exons = analyze_exons(f"{prefix}/{phylum}/{org_name_i}/{postfix}/{nof}")
        exon_37_idx = df_exons[df_exons["length"] == 37].index[0]
        exon_110_idx = exon_37_idx - 1
        data[f"{org_name_i}"] = (df_exons, [exon_110_idx, exon_37_idx])

    return data, found_org_name_i

def create_alignment(phylum, found_org_name_i, align_type = "cds_cassette"):
    org_names = list(found_org_name_i.keys())
    alignment_dict = dict_align_create(phylum, org_names, align_type)  # CHANGE
    alignment_dict_upd = {
        "_".join(key.split("_")[:-1]).capitalize(): value
        for key, value in alignment_dict.items()
    }  # обновляем названия организмов для красоты в Ugene
    os.makedirs(f"../Alignment/{phylum}", exist_ok=True)
    dict_align_to_fasta(alignment_dict_upd, f"../Alignment/{phylum}/{phylum}_cds_cassette.fa")
    dict_align_to_fasta(alignment_dict_upd, f"../Alignment/{phylum}/{phylum}_cds_cassette.aln")

def create_dict_align_info(sub_phylum_list):
    dict_align_info = {}
    for sub_phylum in sub_phylum_list:
        _, found_org_name_i = find_cassettes(sub_phylum)
        dict_align_info[sub_phylum] = found_org_name_i.keys()
    return dict_align_info

## Actinopterygii

### Загрузка данных и создание кассет

In [4]:
# создаем сабсет таблицы для исследуемой филы
phylum = "Actinopterygii"
df_for_analysis = df_taxonomy[df_taxonomy.taxonomy.str.contains(phylum)] # в этой таблице вручную определяем подпапки из принта следующих 2х команд

In [5]:
# создаем принт таксономии для исследуемой филы
# df_for_analysis.to_csv(f"../{phylum}_taxonomy.tsv", sep="\t", header=False)
# create_taxonomy(f"../{phylum}_taxonomy.tsv")

In [6]:
# создаем словарь taxids для последующей загрузки файлов
sub_phylum_list = ["Otomorpha", "Euteleosteomorpha", "Osteoglossomorpha", "Holostei", "Cladistia"]
taxids = create_taxids(df_for_analysis, sub_phylum_list)

In [7]:
# загружаем файлы через datasets: gene, rna, cds, protein; exons (создаются автоматически после загрузки файлов)

# закомментировать, когда все загрузится!
# download_all_files_ncbi(df, taxids, phylas=list(taxids.keys()))

In [8]:
# определяем, у каких видов больше 1 транскрипта
# запускаем еще раз после прогона следующей ячейки, чтобы убедиться, что все удалилось
species_to_update = check_transcript_count(sub_phylum_list)

In [9]:
# удаляем все транскрипты, кроме 1-го - он, как правило, самый адекватный

# закомментировать, когда все обновится!
# update_data_for_species(species_to_update)

In [10]:
# создаем словарь data_phylum для видов, у которых в экзонах нашелся экзон размером 37 нт для дальнейшего создания кассет
for sub_phylum in sub_phylum_list:
    data, found_org_name_i = find_cassettes(sub_phylum)
    # и сразу создаем кассеты для найденных данных
    introns = create_many_cassettes(sub_phylum, data)
    # и еще создаем выравнивания cds_cassette для всех sub_phylum (на всякий случай)
    create_alignment(sub_phylum, found_org_name_i)

### Анализ интрона для всех выбранных сабфил

In [11]:
# сначала создаем словарь с данными
dict_align_info = create_dict_align_info(sub_phylum_list)

In [12]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(dict_align_info, "cds")
df_cds.equal_to_cds.unique()

array([ True])

In [13]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds

df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(dict_align_info, "cds_cassette")
df_cds_cassette

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
0,Otomorpha,ictalurus_punctatus_11,Ictalurus_punctatus,1104,False,1058,46,3166,110,datasets
1,Otomorpha,chanos_chanos_4,Chanos_chanos,1071,False,1070,1,3568,110,datasets
2,Otomorpha,sinocyclocheilus_rhinocerous_12,Sinocyclocheilus_rhinocerous,1215,False,1061,154,3449,110,datasets
3,Otomorpha,carassius_auratus_9,Carassius_auratus,1221,False,1073,148,3854,110,datasets
4,Otomorpha,denticeps_clupeoides_5,Denticeps_clupeoides,1041,False,1034,7,2629,110,datasets
...,...,...,...,...,...,...,...,...,...,...
68,Euteleosteomorpha,austrofundulus_limnaeus_42,Austrofundulus_limnaeus,1167,False,1145,22,2541,110,datasets
69,Euteleosteomorpha,seriola_lalandi_dorsalis_32,Seriola_lalandi_dorsalis,1185,False,1163,22,2481,110,datasets
70,Osteoglossomorpha,paramormyrops_kingsleyae_0,Paramormyrops_kingsleyae,1254,False,1133,121,2929,110,datasets
71,Osteoglossomorpha,scleropages_formosus_1,Scleropages_formosus,1239,False,1127,112,3412,110,datasets


In [14]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
6,Otomorpha,danio_rerio_1,Danio_rerio,1233,False,1082,151,3995,110,datasets
9,Otomorpha,danio_rerio_2,Danio_rerio,1092,False,1091,1,3580,110,datasets


In [15]:
# удаляем дубликаты

# -----------------
# 6 - danio_rerio_1, nxf1b
# -----------------

indices_to_drop = [6]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [16]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/datasets_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Amphibia

### Загрузка данных и создание кассет

In [17]:
# создаем сабсет таблицы для исследуемой филы
phylum = "Amphibia"
df_for_analysis = df_taxonomy[df_taxonomy.taxonomy.str.contains(phylum)] # в этой таблице вручную определяем подпапки из принта следующих 2х команд

In [18]:
# создаем принт таксономии для исследуемой филы
# df_for_analysis.to_csv(f"../{phylum}_taxonomy.tsv", sep="\t", header=False)
# create_taxonomy(f"../{phylum}_taxonomy.tsv")

In [19]:
# создаем словарь taxids для последующей загрузки файлов
sub_phylum_list = ["Gymnophiona", "Caudata", "Anura"]
taxids = create_taxids(df_for_analysis, sub_phylum_list)

In [20]:
# загружаем файлы через datasets: gene, rna, cds, protein; exons (создаются автоматически после загрузки файлов)

# закомментировать, когда все загрузится!
# download_all_files_ncbi(df, taxids, phylas=list(taxids.keys()))

In [21]:
# определяем, у каких видов больше 1 транскрипта
# запускаем еще раз после прогона следующей ячейки, чтобы убедиться, что все удалилось
species_to_update = check_transcript_count(sub_phylum_list)

In [22]:
# удаляем все транскрипты, кроме 1-го - он, как правило, самый адекватный

# закомментировать, когда все обновится!
# update_data_for_species(species_to_update)

In [23]:
# создаем словарь data_phylum для видов, у которых в экзонах нашелся экзон размером 37 нт для дальнейшего создания кассет
for sub_phylum in sub_phylum_list:
    data, found_org_name_i = find_cassettes(sub_phylum)
    # и сразу создаем кассеты для найденных данных
    introns = create_many_cassettes(sub_phylum, data)
    # и еще создаем выравнивания cds_cassette для всех sub_phylum (на всякий случай)
    create_alignment(sub_phylum, found_org_name_i)

### Анализ интрона для всех выбранных сабфил

In [24]:
# сначала создаем словарь с данными
dict_align_info = create_dict_align_info(sub_phylum_list)

In [25]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(dict_align_info, "cds")
df_cds.equal_to_cds.unique()

array([ True])

In [26]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds

df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(dict_align_info, "cds_cassette")
df_cds_cassette

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
0,Gymnophiona,geotrypetes_seraphini_2,Geotrypetes_seraphini,894,False,839,55,3065,110,datasets
1,Gymnophiona,microcaecilia_unicolor_0,Microcaecilia_unicolor,1116,False,929,187,2784,110,datasets
2,Gymnophiona,rhinatrema_bivittatum_1,Rhinatrema_bivittatum,1134,False,1031,103,4053,110,datasets
3,Caudata,ambystoma_mexicanum_0,Ambystoma_mexicanum,1068,False,1067,1,10340,110,datasets
4,Caudata,pleurodeles_waltl_1,Pleurodeles_waltl,1344,False,1193,151,3245,110,datasets
5,Anura,rana_temporaria_0,Rana_temporaria,1017,False,1007,10,3036,110,datasets
6,Anura,bufo_gargarizans_12,Bufo_gargarizans,1017,False,1010,7,2879,110,datasets
7,Anura,xenopus_laevis_18,Xenopus_laevis,1062,False,1010,52,3791,110,datasets
8,Anura,hyla_sarda_10,Hyla_sarda,1032,False,1007,25,3029,110,datasets
9,Anura,xenopus_tropicalis_2,Xenopus_tropicalis,1164,False,1124,40,1714,110,datasets


In [27]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
6,Anura,bufo_gargarizans_12,Bufo_gargarizans,1017,False,1010,7,2879,110,datasets
7,Anura,xenopus_laevis_18,Xenopus_laevis,1062,False,1010,52,3791,110,datasets
9,Anura,xenopus_tropicalis_2,Xenopus_tropicalis,1164,False,1124,40,1714,110,datasets
12,Anura,bufo_gargarizans_13,Bufo_gargarizans,1104,False,869,235,652,110,datasets
13,Anura,xenopus_laevis_17,Xenopus_laevis,1146,False,1073,73,3515,110,datasets
14,Anura,bufo_bufo_15,Bufo_bufo,1089,False,854,235,618,110,datasets
17,Anura,bufo_bufo_14,Bufo_bufo,720,False,713,7,3002,110,datasets
18,Anura,xenopus_tropicalis_1,Xenopus_tropicalis,1083,False,1037,46,2596,110,datasets


In [28]:
# удаляем дубликаты

# -----------------
# 14 - bufo_bufo_15
# 12 - bufo_gargarizans_13
# 13 - xenopus_laevis_17
# 9 - xenopus_tropicalis_2
# -----------------

indices_to_drop = [14, 12, 13, 9]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [29]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/datasets_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Sauropsida

### Загрузка данных и создание кассет

In [30]:
# создаем сабсет таблицы для исследуемой филы
phylum = "Sauropsida"
df_for_analysis = df_taxonomy[df_taxonomy.taxonomy.str.contains(phylum)] # в этой таблице вручную определяем подпапки из принта следующих 2х команд

In [31]:
# создаем принт таксономии для исследуемой филы
# df_for_analysis.to_csv(f"../{phylum}_taxonomy.tsv", sep="\t", header=False)
# create_taxonomy(f"../{phylum}_taxonomy.tsv")

In [32]:
# создаем словарь taxids для последующей загрузки файлов
sub_phylum_list = ["Lepidosauria", "Testudinata", "Crocodylia", "Aves"]
taxids = create_taxids(df_for_analysis, sub_phylum_list)

In [33]:
# загружаем файлы через datasets: gene, rna, cds, protein; exons (создаются автоматически после загрузки файлов)

# закомментировать, когда все загрузится!
# download_all_files_ncbi(df, taxids, phylas=list(taxids.keys()))

In [34]:
# определяем, у каких видов больше 1 транскрипта
# запускаем еще раз после прогона следующей ячейки, чтобы убедиться, что все удалилось
species_to_update = check_transcript_count(sub_phylum_list)

# удалил Aves/anas_platyrhynchos_45, т.к. там ничего не скачалось

In [35]:
# удаляем все транскрипты, кроме 1-го - он, как правило, самый адекватный

# закомментировать, когда все обновится!
# update_data_for_species(species_to_update)

In [36]:
# создаем словарь data_phylum для видов, у которых в экзонах нашелся экзон размером 37 нт для дальнейшего создания кассет
for sub_phylum in sub_phylum_list:
    data, found_org_name_i = find_cassettes(sub_phylum)
    # и сразу создаем кассеты для найденных данных
    introns = create_many_cassettes(sub_phylum, data)
    # и еще создаем выравнивания cds_cassette для всех sub_phylum (на всякий случай)
    create_alignment(sub_phylum, found_org_name_i)

### Анализ интрона для всех выбранных сабфил

In [37]:
# сначала создаем словарь с данными
dict_align_info = create_dict_align_info(sub_phylum_list)

In [38]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(dict_align_info, "cds")
df_cds.equal_to_cds.unique()

array([ True])

In [39]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds

df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(dict_align_info, "cds_cassette")
df_cds_cassette

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source
0,Lepidosauria,python_bivittatus_26,Python_bivittatus,1035,False,1034,1,2374,110,datasets
1,Lepidosauria,notechis_scutatus_22,Notechis_scutatus,1023,False,1022,1,2507,110,datasets
2,Lepidosauria,pseudonaja_textilis_21,Pseudonaja_textilis,1029,False,1028,1,2519,110,datasets
3,Lepidosauria,anolis_sagrei_5,Anolis_sagrei,1038,False,1037,1,4667,110,datasets
4,Lepidosauria,pituophis_catenifer_annectens_0,Pituophis_catenifer_annectens,1320,False,1319,1,2420,110,datasets
...,...,...,...,...,...,...,...,...,...,...
76,Aves,pezoporus_wallicus_18,Pezoporus_wallicus,1572,False,1004,568,1328,110,datasets
77,Aves,aphelocoma_coerulescens_4,Aphelocoma_coerulescens,1077,False,992,85,3626,110,datasets
78,Aves,catharus_ustulatus_26,Catharus_ustulatus,1227,False,986,241,3252,110,datasets
79,Aves,chroicocephalus_ridibundus_12,Chroicocephalus_ridibundus,1566,False,1004,562,1373,110,datasets


In [40]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,sub_phylum,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length,source


In [41]:
# удаляем дубликаты

# -----------------

# -----------------

# indices_to_drop = []
# df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [42]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/datasets_{phylum}_cds_cassette.tsv", sep="\t", index=False)